In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:

import os
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.dashscope import DashScopeEmbedding, DashScopeTextEmbeddingModels

# 增加调试日志
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger("llama_index").addHandler(logging.StreamHandler(stream=sys.stdout))


Settings.llm = OpenAILike(
    model="qwen-plus",
    api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    is_chat_model=True
)

Settings.embed_model = DashScopeEmbedding(
    model_name=DashScopeTextEmbeddingModels.TEXT_EMBEDDING_V3,
    embed_batch_size=6,
    embed_input_length=8192
)

documents = SimpleDirectoryReader("data").load_data()



None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


> [SimpleDirectoryReader] Total files added: 1
DEBUG:llama_index.core.readers.file.base:> [SimpleDirectoryReader] Total files added: 1
DEBUG:fsspec.local:open file: C:/Users/LiuHuijun/PycharmProjects/ai-study/week03/code/data/公司员工休假福利管理制度.txt


In [3]:
print(documents)

[Document(id_='c8c842a2-0851-47d1-a00e-69f7caea61df', embedding=None, metadata={'file_path': 'C:\\Users\\LiuHuijun\\PycharmProjects\\ai-study\\week03\\code\\data\\公司员工休假福利管理制度.txt', 'file_name': '公司员工休假福利管理制度.txt', 'file_type': 'text/plain', 'file_size': 6163, 'creation_date': '2026-01-14', 'last_modified_date': '2026-01-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='# 公司员工休假福利管理制度\n\n## 第一章 总则\n\n**第一条 目的**  \n为规范公司员工休假管理，保障员工合法权益，维护正常的工作秩序，提升员工满意度与工作效率，根据《中华人民共和国劳动法》《职工带薪年休假条例》《女职工劳动保护特别规定》等国家相关法律法规，结合公司实际情况，制定本制度。\n\n**第二条 适用范围**  \n本制度适用于公司所有与公司签订劳动合同的正式员工（含试用期员工），劳务派遣人员参照执行，具体以劳务派遣协议为准。\n\n**第三条 基本原则**  \n1. 依法合规：严格遵守国家法律法规及相关政

In [5]:
# 添加一个新的 Document
from llama_index.core import Document
text = "CEO 可以直接请假，无需向直接领导汇报"

doc = Document(
    text = text,
    metadata = {
        "author": "wilson yin",
        "title": "CEO 请假申请",
        "id": "1234567890"
    }
)

In [6]:
print(doc)

Doc ID: 8a00d453-77dc-43d8-bfae-6388ec776ab8
Text: CEO 可以直接请假，无需向直接领导汇报


In [7]:
#  手动切分Documents

from  llama_index.core.schema import TextNode

n1 = TextNode(text=doc.text[0:8],doc_id=doc.id_)
n2 = TextNode(text=doc.text[9:16],doc_id=doc.id_)

print(n1)
print(n2)

Node ID: 0980503f-5b5b-4260-8e18-17ef0ccb820a
Text: CEO 可以直接
Node ID: 9ad69e57-6d5c-4e7d-8317-dde8470fa6a1
Text: 假，无需向直接


In [8]:
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core import Document

doc = Document(
    text=("""
    ### 第七条 事假  
    1. 员工因私事必须本人处理的，可申请事假。  
    2. 事假需提前申请并获直属主管批准，紧急情况可事后补办手续。  
    3. 事假为无薪假，按日扣除相应工资。  
    4. 每月事假原则上不超过3天，全年累计不超过15天，特殊情况需经人力资源部及公司领导审批。
    """
    ),
    metadata={"title": "Vacation Questions"}
)

# 内置切分器

splitter = TokenTextSplitter(
    chunk_size=64,
    chunk_overlap=4,
    separator="\n"
)

nodes = splitter.get_nodes_from_documents([doc])

for node in nodes:
    print(node.text)
    print(node.metadata)


> Adding chunk: ### 第七条 事假  
    1. 员工因私事必须本人处理的，可申请事假。
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: ### 第七条 事假  
    1. 员工因私事必须本人处理的，可申请事假。
> Adding chunk: 2. 事假需提前申请并获直属主管批准，紧急情况可事后补办手续。
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 2. 事假需提前申请并获直属主管批准，紧急情况可事后补办手续。
> Adding chunk: 3. 事假为无薪假，按日扣除相应工资。
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 3. 事假为无薪假，按日扣除相应工资。
> Adding chunk: 4. 每月事假原则上不超过3天，全年累计不超过15天，特殊情况需经人力资源部及公司领导审批。
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 4. 每月事假原则上不超过3天，全年累计不超过15天，特殊情况需经人力资源部及公司领导审批。
### 第七条 事假  
    1. 员工因私事必须本人处理的，可申请事假。
{'title': 'Vacation Questions'}
2. 事假需提前申请并获直属主管批准，紧急情况可事后补办手续。
{'title': 'Vacation Questions'}
3. 事假为无薪假，按日扣除相应工资。
{'title': 'Vacation Questions'}
4. 每月事假原则上不超过3天，全年累计不超过15天，特殊情况需经人力资源部及公司领导审批。
{'title': 'Vacation Questions'}
